In [7]:
import pandas as pd

# Load dataset
df = pd.read_csv("/Users/badekale/Documents/cop_summary/Sheet 5-Table 1/Compiled evaluation result.csv")

# Define relevant columns
relevant_columns = ['Gemma2_2b_faithfulness', 'Mistral_7b_faithfulness',
                   'Gemma2_2b_answer_relevancy', 'Mistral_7b_answer_relevancy',
                   'Gemma2_2b_context_utilization', 'Mistral_7b_context_utilization',
                   'Human_faithfulness', 'Human_answer_relevancy', 'Human_context_utilization']

# Compute column means and standard deviations, ignoring 0s
column_means = df[relevant_columns].replace(0, pd.NA).mean()
column_stds = df[relevant_columns].replace(0, pd.NA).std()

# Combine means and standard deviations into a dictionary for display
stats = {col: (mean, std) for col, mean, std in zip(relevant_columns, column_means, column_stds)}

# Display results
print("\n**Mean Scores and Standard Deviations for Each Metric:**\n")
for metric, (mean, std) in stats.items():
    print(f"{metric}: Mean = {mean:.3f}, Std = {std:.3f}")


**Mean Scores and Standard Deviations for Each Metric:**

Gemma2_2b_faithfulness: Mean = 0.966, Std = 0.102
Mistral_7b_faithfulness: Mean = 0.848, Std = 0.271
Gemma2_2b_answer_relevancy: Mean = 0.888, Std = 0.057
Mistral_7b_answer_relevancy: Mean = 0.893, Std = 0.050
Gemma2_2b_context_utilization: Mean = 0.939, Std = 0.184
Mistral_7b_context_utilization: Mean = 0.980, Std = 0.108
Human_faithfulness: Mean = 0.760, Std = 0.081
Human_answer_relevancy: Mean = 0.993, Std = 0.037
Human_context_utilization: Mean = 0.713, Std = 0.101


In [ ]:
# Load dataset
df = pd.read_csv("/Users/badekale/Documents/cop_summary/Sheet 5-Table 1/Compiled evaluation result.csv")  # Adjust file path

# Define relevant columns
relevant_columns = [
    #'Gemma2_2b_faithfulness', 
    #'Mistral_7b_faithfulness',
    #'Gemma2_2b_answer_relevancy', 
    #'Mistral_7b_answer_relevancy',
    #'Gemma2_2b_context_utilization', 
    'Mistral_7b_context_utilization',
    #'Human_faithfulness', 
    #'Human_answer_relevancy', 
    'Human_context_utilization'
]

# Identify rows that contain at least one NaN or zero
mask = df[relevant_columns].isna().any(axis=1) | (df[relevant_columns] == 0).any(axis=1)

num_identified_rows = mask.sum()
print(f"Number of rows identified: {num_identified_rows}")

# Replace entire rows with zeros where any NaN or zero exists
df.loc[mask, relevant_columns] = 0

# ========================= CASE 1: No Forward-Fill =========================
df_no_fill = df[relevant_columns].copy()

# ========================= CASE 2: Replace NaN with Column Mean (Ignoring Zeros) =========================
# Convert zeros to NaN so they don't affect the mean calculation
df_ignore_zeros = df[relevant_columns].replace(0, pd.NA)

# Compute column means ignoring NaNs (and previous zeros)
column_means_ignore_zeros = df_ignore_zeros.mean()

# Permanently replace NaNs (including previous zeros) with computed means
df_ignore_zeros = df_ignore_zeros.fillna(column_means_ignore_zeros)

# ========================= CASE 3: Replace NaN and Zero with Column Mean (Including Zeros) =========================
df_include_zeros = df[relevant_columns].copy()

# Compute mean including zeros
column_means_include_zeros = df_include_zeros.mean()

# Replace all NaN and zero values with column means
df_include_zeros = df_include_zeros.apply(lambda col: col.replace(0, column_means_include_zeros[col.name]).fillna(column_means_include_zeros[col.name]), axis=0)

# ========================= Step 4: Compute Spearman Rank Correlation =========================
spearman_corr_no_fill = df_no_fill.corr(method='spearman')
spearman_corr_ignore_zeros = df_ignore_zeros.corr(method='spearman')
spearman_corr_include_zeros = df_include_zeros.corr(method='spearman')

# ========================= Step 5: Extract Key Correlations =========================
key_metrics = [
    #('Gemma2_2b_faithfulness', 'Mistral_7b_faithfulness'),
    #('Gemma2_2b_answer_relevancy', 'Mistral_7b_answer_relevancy'),
    #('Gemma2_2b_context_utilization', 'Mistral_7b_context_utilization'),
    #('Human_faithfulness', 'Gemma2_2b_faithfulness'),
    #('Human_answer_relevancy', 'Gemma2_2b_answer_relevancy'),
    #('Human_context_utilization', 'Gemma2_2b_context_utilization'),
    #('Human_faithfulness', 'Mistral_7b_faithfulness'),
    #('Human_answer_relevancy', 'Mistral_7b_answer_relevancy'),
    ('Human_context_utilization', 'Mistral_7b_context_utilization')
]

# ========================= Step 6: Print Key Correlations =========================
def print_key_correlations(correlation_matrix, case_name):
    print(f"\n**Spearman Rank Correlations - {case_name}:**\n")
    for metric1, metric2 in key_metrics:
        print(f"{metric1} vs {metric2}: {correlation_matrix.loc[metric1, metric2]:.3f}")

# Print correlations for each case
print_key_correlations(spearman_corr_no_fill, "No Forward Fill")
print_key_correlations(spearman_corr_ignore_zeros, "Mean Replacement (Ignoring Zeros)")
print_key_correlations(spearman_corr_include_zeros, "Mean Replacement (Including Zeros)")


Number of rows identified: 1

**Spearman Rank Correlations - No Forward Fill:**

Human_context_utilization vs Mistral_7b_context_utilization: 0.401

**Spearman Rank Correlations - Mean Replacement (Ignoring Zeros):**

Human_context_utilization vs Mistral_7b_context_utilization: 0.280

**Spearman Rank Correlations - Mean Replacement (Including Zeros):**

Human_context_utilization vs Mistral_7b_context_utilization: 0.280


/var/folders/ps/_j2fnmks75s10c6kq1tt3gkw0000gn/T/ipykernel_45601/3368968520.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_ignore_zeros = df_ignore_zeros.fillna(column_means_ignore_zeros)
